# `vegas` library

Summary of some selected sections of `vegas` [documentation](https://vegas.readthedocs.io/en/latest/index.html)

## Tutorial

### Introduction

This library provides a Monte Carlo estimate of multidimensional integrals.
The algorithm has two components:
- Automatic variable transformation in order to flatten the integrand
- Monte Carlo estimate of the flattened integral

The above two steps are iterated in order to optimize the variable transformation and collect information about the integrand.

There are very few assumptions about the integrand: it doesn't need to be analytical, nor continuous.

The algorithm is adaptive (two strategies are used: importance sampling and adaptive stratified sampling), meaning it is able to catch irregularities in the multidimensional space.

Each Monte Carlo estimate of the integral can be considered as drawn from a Gaussian distribution where the mean is the actual value of the integral.

Error analysis is straightforward, if we generate multiple Monte Carlo estimates of the integral.

###  Basic integrals

Let's try and estimate the following integral (with $C$ chosen so that the exact integral is exactly $1$, and we can see if the estimates are correct):
$$ C \int_{-1}^{1} dx_0 \int_0^1 dx_1 \int_0^1 dx_2 \int_0^3 dx_3 e^{-100\sum_d (x_d - 0.5)^2} = 1$$

In [22]:
import vegas
import math

def f(x):
    dx2 = 0
    for d in range(4):
        dx2 += (x[d] - 0.5) ** 2
    return math.exp(-dx2 * 100.) * 1013.2118364296088

integ = vegas.Integrator([[-1, 1], [0, 1], [0, 1], [0, 1]])

result = integ(f, nitn=10, neval=1000)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   1.15(87)        1.15(87)            0.00     1.00
  2   1.23(42)        1.22(38)            0.01     0.93
  3   1.09(12)        1.10(11)            0.06     0.95
  4   1.076(59)       1.081(52)           0.05     0.99
  5   1.018(34)       1.037(28)           0.30     0.88
  6   1.023(24)       1.029(18)           0.26     0.93
  7   1.015(19)       1.022(13)           0.26     0.95
  8   0.995(14)       1.0098(97)          0.50     0.83
  9   1.010(13)       1.0100(78)          0.44     0.90
 10   0.990(12)       1.0039(65)          0.62     0.78

result = 1.0039(65)    Q = 0.78


- `nitn`: number of iterations of `vegas` algorithm
- `neval`: number of evaluations of the integrand for each `vegas` iteration

Let's make some observations about the above result

#### Adaptation
For each of the 10 `vegas` iterations, the algorithm makes an estimate of the integral, and then it performs a weighted (based on uncertainty) average to give a best estimate of the integral using all the information gathered so far.

In the first iterations, the uncertainty is high because the algorithm has no information about the integrand.

After each iteration, the integration variables are re-mapped for subsequent iterations, concentrating more samples where the function value is larger, thus decreasing the error.

Eventually, the error stops decreasing because the algorithm has fully adapted to the integrand.

#### Weighted average
The notation $0.0009779(67)$ should be read as:
$$0.0009779(67) \equiv 0.0009779 \pm 0.0000067$$

The individual `integral` values are samples from a Gaussian distribution (provided `neval` is sufficiently large) whose mean is the exact value of the integral, and the error represents the estimate of the standard deviation of such distribution.

The weighted average $\bar{I}$ minimizes:
$$\chi^2 = \sum_i \frac{(I_i - \bar{I})^2}{\sigma_i^2}$$
where $I_i \pm \sigma_i$ are the estimates from individual iterations.

If $I_i$ are Gaussian, the $\chi^2$ should be of order the number of degrees of freedom (`dofs = neval - 1`) plus or minus the square root of double the dofs.

The distributions, however, are likely not Gaussian, and if $\chi^2$ is much larger than the number of dofs, then the error analysis in unreliable.

To quantify this unreliability, we need to look at the `Q` or *p-value* of the $\chi^2$, that is the probability that a larger $\chi^2$ could result from Gaussian fluctuations.

If `Q` is too small (below $0.05$ or $0.1$), it means that the large value of $\chi^2$ is not random, but rather caused by an insufficient number of iterations `neval`.

##### `RAvg` object
`integ()` returns a `RAvg` object, which has the following attributes:
- `mean`: weighted average of all estimates of the integral
- `sdev`: standard deviation of the weighted average
- `chi2`: $\chi^2$ of the weighted average
- `dof`: the number of degrees of freedom
- `Q`: the *p-value* of the weighted average's $\chi^2$
- `itn_results`: list of the integral estimates from each iteration
- `sum_neval`: total number of single integral evaluations
- `avg_neval`: average number of integral evaluations per iteration

#### Precision
The precision is determined by `nitn` and `neval`. The computational cost is proportional to `nitn * neval`.

The number of single integral evaluations varies between `vegas` iterations, being maximum in the first ones.

To increase precision, one should typically increase `neval`, and then `nitn`.

In [23]:
result = integ(f, nitn=100, neval=1000)
print('larger nitn  => %s    Q = %.2f' % (result, result.Q))

result = integ(f, nitn=10, neval=1e4)
print('larger neval => %s    Q = %.2f' % (result, result.Q))

larger nitn  => 1.0004(13)    Q = 0.27
larger neval => 0.99958(52)    Q = 0.14


In the above example, increasing `neval` leads to a more accurate result.

Typically, no more than 10 or 20 iterations beyond the point where `vegas` has fully adapted are needed, just to check the values of $\chi^2$ and $Q$. It is useful to compare results where `neval` differs by a factor of $4-10$, which should agree within errors. If they do not, it means that `neval` is too small.

`vegas` has two sources of error:
- statistical error (goes as 1/sqrt(`neval`))
- systematic error due to non-Gaussian effects (goes as 1/`neval`, and becomes negligible wrt statistical error as `neval` increases), that can be spotted by changing `neval` and seeing that results are not compatible within errors

Making `nitn` larger keeping `neval` fixed is evantually deemed to give wrong results, because the statistical error will not mask the systematic one anymore.

#### Early iterations
With very peaky integrands, the early iterations are very far from the correct answer, with unrealiable results and error estimates.

In [24]:
integ = vegas.Integrator([[-2, 2], [0, 2], [0, 2], [0., 2]])
result = integ(f, nitn=10, neval=1000)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   0.0055(49)      0.0055(49)          0.00     1.00
  2   2.8(1.7)        0.0055(49)          2.72     0.10
  3   2.6(1.2)        0.0055(49)          3.77     0.02
  4   1.09(21)        0.0061(49)         11.07     0.00
  5   0.998(74)       0.0104(49)         52.46     0.00
  6   1.063(37)       0.0282(49)        197.37     0.00
  7   1.009(23)       0.0718(48)        465.94     0.00
  8   0.985(19)       0.1267(46)        716.60     0.00
  9   1.021(15)       0.2000(44)       1012.58     0.00
 10   1.018(13)       0.2818(42)       1281.25     0.00

result = 0.2818(42)    Q = 0.00


`vegas` misses the peak completely in the first iterations, and so they pollute the error analysis, giving $Q = 0.00$.

This is solved by letting `vegas` run discarding results for a few iterations, letting it adapt to the integrand, and then do the error analysis:

In [25]:
integ = vegas.Integrator([[-2, 2], [0, 2], [0, 2], [0., 2]])

# step 1 -- adapt to f; discard results
integ(f, nitn=10, neval=1000)

# step 2 -- integ has adapted to f; keep results
result = integ(f, nitn=10, neval=1000)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   0.983(13)       0.983(13)           0.00     1.00
  2   0.967(14)       0.9760(94)          0.70     0.40
  3   1.012(11)       0.9912(71)          3.41     0.03
  4   1.026(15)       0.9973(65)          3.64     0.01
  5   0.995(16)       0.9970(60)          2.73     0.03
  6   1.013(19)       0.9984(57)          2.31     0.04
  7   1.009(14)       0.9999(53)          2.01     0.06
  8   1.006(15)       1.0005(50)          1.74     0.10
  9   1.009(14)       1.0014(47)          1.56     0.13
 10   1.000(13)       1.0013(44)          1.39     0.19

result = 1.0013(44)    Q = 0.19


#### Other integrands
Once `integ` has been trained on an integrand, it can be successfully used on other functions with similar structure (peaks in the same region).

In [26]:
def g(x):
    return x[0] * f(x)

result = integ(g, nitn=10, neval=1000)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   0.5026(62)      0.5026(62)          0.00     1.00
  2   0.4900(59)      0.4960(43)          2.15     0.14
  3   0.4979(70)      0.4966(37)          1.10     0.33
  4   0.4971(85)      0.4966(34)          0.74     0.53
  5   0.5078(83)      0.4982(31)          0.94     0.44
  6   0.4873(75)      0.4966(29)          1.11     0.35
  7   0.4959(73)      0.4965(27)          0.92     0.48
  8   0.510(12)       0.4972(26)          0.96     0.46
  9   0.5057(86)      0.4979(25)          0.95     0.47
 10   0.509(10)       0.4986(24)          0.98     0.45

result = 0.4986(24)    Q = 0.45


#### Non-rectangular volumes
`vegas` can be used to integrate over non-rectangular domains, by setting the value of the function outside the integration volume equal to zero.

In [27]:
import vegas
import math

def f_sph(x):
    dx2 = 0
    for d in range(4):
        dx2 += (x[d] - 0.5) ** 2
    if dx2 < 0.2 ** 2:
        return math.exp(-dx2 * 100.) * 1115.3539360527281318 # to make exact integral equal 1
    else:
        return 0.0

integ = vegas.Integrator([[-1, 1], [0, 1], [0, 1], [0, 1]])

integ(f_sph, nitn=10, neval=1000)           # adapt the grid
result = integ(f_sph, nitn=10, neval=1000)  # estimate the integral
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   0.995(20)       0.995(20)           0.00     1.00
  2   1.003(21)       0.999(15)           0.09     0.76
  3   0.930(15)       0.965(10)           5.45     0.00
  4   0.972(21)       0.9663(94)          3.67     0.01
  5   1.013(41)       0.9687(91)          3.06     0.02
  6   1.009(35)       0.9713(88)          2.70     0.02
  7   1.071(64)       0.9731(87)          2.65     0.01
  8   1.027(30)       0.9773(84)          2.69     0.01
  9   0.998(26)       0.9792(80)          2.43     0.01
 10   0.988(20)       0.9805(74)          2.18     0.02

result = 0.9805(74)    Q = 0.02


It is a good practice to make the integration volume as large as possible with respect to the rectangle passed to `vegas`.

Integrating to infinity is possible by changing variables (using, e.g. $x = bz/(1-z)$ or $\arctan$)

#### Damping
One can slow down the `vegas` process of adaptation to reduce the error fluctuations between iterations, using `alpha`, which defaults to $0.5$.

In [28]:
integ = vegas.Integrator([[-1, 1], [0, 1], [0, 1], [0, 1]])
integ(f_sph, nitn=10, neval=1000, alpha=0.1)
result = integ(f_sph, nitn=10, neval=1000, alpha=0.1)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   1.029(26)       1.029(26)           0.00     1.00
  2   0.979(23)       1.001(17)           2.04     0.15
  3   0.977(22)       0.992(14)           1.41     0.24
  4   1.035(21)       1.005(11)           1.96     0.12
  5   0.982(20)       0.9993(99)          1.71     0.15
  6   0.984(20)       0.9963(89)          1.46     0.20
  7   1.010(18)       0.9990(80)          1.30     0.25
  8   1.011(17)       1.0010(73)          1.17     0.32
  9   0.996(17)       1.0003(67)          1.03     0.41
 10   0.983(17)       0.9980(62)          1.02     0.42

result = 0.9980(62)    Q = 0.42


If there are persistent, large fluctuations in the size of the per-iteration errors, then `alpha` should be reduced.

With larger `alpha`s, `vegas` can over-react to random fluctuations it encounters as it samples the integrand.

We want `alpha` to be large enough so that `vegas` gives the result as quickly as possible, but not too large to make it fluctuate too much once it finds the optimal value.

#### `adapt=False`
One can switch off adaptation for three reasons:
- Instability in error fluctuations (as described in previous paragraph)
- `vegas` runs faster when it is not adapting, so it can be useful for simple integrands
- When adaptation is off, `vegas` uses unweighted averages instead of weighted ones, and the averages have no systematic error, giving correct results even for large `nitn`

Switching off adaptation is useful when `neval` is too small for the integrand.

One can leave adaptation on for the first 10 iterations, and then switch it off for the following ones, and perform a statistical analysis (in this case, to use the adaptation, `neval` must not change between the two runs).

### Faster integrands
In some cases, it may be necessary to send `neval` to a very big number $10^5$ or more.

We can use the batch mode of `vegas`.

In [30]:
import vegas
import numpy as np

@vegas.lbatchintegrand
def f_batch(x):
    # evaluate integrand at multiple points simultaneously
    dim = x.shape[1] # instead of len(x)
    norm = 1013.2118364296088 ** (dim / 4.)
    dx2 = 0.0
    for d in range(dim):
        dx2 += (x[:, d] - 0.5) ** 2   # instead of dx2 += (x[d] - 0.5) ** 2
    return np.exp(-100. * dx2) * norm

integ = vegas.Integrator(4 * [[0, 1]])

integ(f_batch, nitn=10, neval=2e5)
result = integ(f_batch, nitn=10, neval=2e5)
print('result = %s   Q = %.2f' % (result, result.Q))

result = 0.999970(78)   Q = 0.38


The function `f_batch` takes an array of multidimensional points `x[i, d]` (where `i` labels the point and `d` the point component) instead of a single point and returns an array of function values. The decorator `vegas.llbatchintegrand` makes `vegas` pass a batch of sample points to evaluate the integral, and this speeds up the integration.

An alternative to decorate the function is to decorate the class (or extend `vegas.LBatchIntegrand`):

In [31]:
import vegas
import numpy as np

@vegas.lbatchintegrand
class f_batch:
    def __init__(self, dim):
        self.dim = dim
        self.norm = 1013.2118364296088 ** (dim / 4.)

    def __call__(self, x):
        # evaluate integrand at multiple points simultaneously
        dx2 = 0.0
        for d in range(self.dim):
            dx2 += (x[:, d] - 0.5) ** 2
        return np.exp(-100. * dx2) * self.norm

f = f_batch(dim=4)
integ = vegas.Integrator(f.dim * [[0, 1]])

integ(f, nitn=10, neval=2e5)
result = integ(f, nitn=10, neval=2e5)
print('result = %s   Q = %.2f' % (result, result.Q))

result = 1.000139(78)   Q = 0.25


As an alternative, one can write the integrand in Cython (see `vegas` docs).

### Multiple processors

In [33]:
import vegas
import numpy as np

# Integrand: ridge of N Gaussians spread along part of the diagonal
def ridge(x):
    N = 10000
    x0 = np.linspace(0.4, 0.6, N)
    dx2 = 0.0
    for xd in x:
        dx2 += (xd - x0) ** 2
    return np.average(np.exp(-100. * dx2)) *  (100. / np.pi) ** (len(x) / 2.)

def main():
    integ = vegas.Integrator(4 * [[0, 1]], nproc=8)  # 8 processors
    # adapt
    integ(ridge, nitn=10, neval=1e4)
    # final results
    result = integ(ridge, nitn=10, neval=1e4)
    print('result = %s    Q = %.2f' % (result, result.Q))

if __name__ == '__main__':
    main()

The code above will generate an AttributeError when run in some interactive environments (as opposed to running from the command line) on some platforms. This can usually be fixed by putting the integrand function ridge(x) into a file and importing it into the script.

One can use Python's `multiprocessing` module

In [ ]:
import multiprocessing
import numpy as np
import vegas

class parallelintegrand(vegas.BatchIntegrand):
    """ Convert (batch) integrand into multiprocessor integrand.

    Integrand should return a numpy array.
    """
    def __init__(self, fcn, nproc=4):
        " Save integrand; create pool of nproc processes. "
        super().__init__()
        self.fcn = fcn
        self.nproc = nproc
        self.pool = multiprocessing.Pool(processes=nproc)
    def __del__(self):
        " Standard cleanup. "
        self.pool.close()
        self.pool.join()
    def __call__(self, x):
        " Divide x into self.nproc chunks, feeding one to each process. "
        nx = x.shape[0] // self.nproc + 1
        # launch evaluation of self.fcn for each chunk, in parallel
        results = self.pool.map(
            self.fcn,
            [x[i*nx : (i+1)*nx] for i in range(self.nproc)],
            1,
            )
        # convert list of results into a single numpy array
        return np.concatenate(results)

Then `fparallel = parallelintegrand(f, 4)`, for example, will create a new integrand `fparallel(x)` that uses 4 CPU cores.